In [1]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from crvusdsim.pool import get
from src.sim.scenario import Scenario
from src.utils.poolgraph import PoolGraph
from src.configs.tokens import *


%load_ext autoreload
%autoreload 2

In [2]:
# Generate markets and prices
scenario = Scenario("baseline")
sample = scenario.pricepaths[0]
scenario.update_market_prices(sample)

# Unpack
markets = scenario.markets
arbitrageur = scenario.arbitrageur
llamma = scenario.llamma
controller = scenario.controller
peg_keepers = scenario.peg_keepers

[DEBUG][23:08:36][root]-685605: Using 567360 1Inch quotes.


In [3]:
prices = scenario.pricepaths.prices
prices.head()

,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xdac17f958d2ee523a2206206994597c13d831ec7,0x8e870d67f660d95d5be530380d0ec0bd388289e1,0x0000000000085d4780b73119b644ae5ecd22b376,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,0xac3e018457b222d93114458476f3e3416abbe38f,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0x18084fba666a33d37592fa2633fd49a74dd93a88
2023-12-03 23:08:28,0.999901,0.999365,1.001000,0.999899,2202.630000,2527.140000,2342.690000,40113.000000,39999.000000
2023-12-04 00:08:28,1.001346,0.999089,1.000454,0.998865,2199.236391,2521.340457,2340.039421,40043.677333,39918.292809
2023-12-04 01:08:28,1.000511,1.000338,0.999089,0.997782,2197.805600,2518.584887,2340.021821,40033.950278,39957.802260
2023-12-04 02:08:28,1.000536,1.000730,0.999323,0.998366,2202.176690,2521.318015,2342.953606,40059.733634,40009.675182
2023-12-04 03:08:28,1.000977,0.999307,0.999105,0.997853,2203.788141,2524.710264,2346.543222,40051.888044,40033.110445


In [4]:
prices.index[0].timestamp()

1701644908.0

In [15]:
prices[[a.lower() for a in llamma.assets.addresses]]

KeyError: "['0xf939e0a03fb07f59a73314e73794be0e57ac1b4e'] not in index"

In [ ]:
# Currently only simulating one LLAMMA. TODO simulate multiple LLAMMAs
scenario.prepare_for_run(scenario.pricepaths[0])

for ts, sample in scenario.pricepaths:
    scenario.update_market_prices(sample)
    scenario.prepare_for_trades(ts)
    scenario.perform_actions(sample)
    scenario.after_trades()
    scenario.update_metrics()